### Staging Changes
Before we can commit changes, we need to add them to staging area (also know as cache or index in legacy versions). A change can be file modification, addition or deletion. Examples:  

To add a file to staging area,
```sh
git add main.py
```

To add only modified (and deleted) files,
```sh
git add -u
```

Newly added files are not tracked by git. To include untracked files as well,
```sh
git add -A
```
The above command will add all modified, new or deleted files to the staging area. If we want to limit the above command to the current directory only,
```sh
git add .
```

What if we make some more changes to the file after staging? In git, files (per se) aren't staged, changes are. So when you do `git add <filename>`, it is the current state of that file that is staged. If you change the file and want to add those changes as well, you just do another `git add`.

To remove a file from the staging area,
```sh
git reset HEAD -- main.py
```

### Commiting Changes
The next step after staging is commiting changes to the local repository. Commit does not make any change to remote repository. To commit changes,
```sh
git commit -m "Commit message here"
```
![Git Commit Workflow](https://i.imgur.com/XTgcp19.jpg)  
  
Once a commit is done, the staging area becomes vacant. What if we forgot to include some changes in a commit? We can amend our last commit. For example if we forgot to add util.py to our prevous commit, we can do the following:
```sh
git add util.py
git commit --amend -m "Amended last commit"
```
However in case of multiple users involving remote repositories, it is not advisable to amend commits.  

![Git Commit Amend Workflow](https://i.imgur.com/1OfPtOv.jpg) 

Each commit is represented by SHA1 hash of the commit object.

### Tracking Changes
In context of git, an `untracked file` is a file which is newly added and not being managed by git. If this file is added to staging area (and not removed) at some point, it becomes `tracked`. We use `git status` to see what files are tracked and what tracked files have been modified since last commit. It also lists all untracked files.

Apart from status, we can also use `git diff` to see the difference between content at different levels in local environment. Before we explore status, we should be aware of `HEAD`. HEAD represents the latest commit in the current branch.  

Writing just git diff starts from working directory and compares with staging area. If no changes have been added to the staging area, then the command compares with the local repository (latest commit). If we want to compare changes in staging area to local repository,
```sh
git diff --staged
```

We can also diff between working directory and a particular commit. To see differences with latest commit,
```sh
git diff HEAD
```

We can specify commit SHA1 to compare with a particular commit.
```sh
git diff c25a62d 
```

We can also compare changes between two commits.
```sh
git diff c25a62d fc5c99f
```

To ignore whitespace differences,
```sh
git diff -w
```

Once we have enough number of commits we may need to go through all the commits done. To view all commits done in the current branch,
```sh
git log
```

To see a more compact version,
```sh
git log --oneline
```

To also see changes (similar to git diff) that were done in each commit,
```sh
git log -p
```

To see all commits by a particular user,
```sh
git log --author="User"
```

To see all commits where a particular file was involved,
```sh
git log util.py
```

### Tags
A tag is just a symbolic name you attach to a specific commit. To tag a commit we need to provide a tag name and commit SHA1. Tag name cannot have whitespace. For example:
```sh
git tag MyCommit fc5c99f
```

![Git Tag](https://i.imgur.com/VkhM4ez.jpg)

We can add more than one tag to a commit. Just write git tag again. To delete tag,
```sh
git tag -d MyCommit
```

If we want a tag to point to a different commit,
```sh
git tag -f MyCommit fc5c99f
```

### Undoing Changes in History
The `reset` command moves HEAD to a previous commit, optionally replacing staging area and working directory with contents of the commit. Reset can be:
- soft—only update the HEAD of the local repository.
- mixed—(default) update the HEAD of the local repository and the staging area.
- hard—update the HEAD of the local repository, the staging area, and the working directory.

Before providing an example of reset, we should know about absolute and relative SHA1. We know that HEAD represents the latest commit. `HEAD^` means one commit before the latest commit. Similarly `HEAD^^` means two commits before HEAD. This can also be written as `HEAD~2`.
```sh
git reset --soft HEAD^
```  

![Git Reset](https://i.imgur.com/RKCLGJ5.png)

The problem with git reset is what of you reset changes you already pushed to remote repository? Other people might have pulled those changes. A push after reset may fail in this case. Therefore it is better to use `git revert`. Reverting creates a new commit which undoes all the changes made earlier.
```sh
git revert HEAD~1
```

### Branching
A branch in git is simply a reference to a commit like a tag difference being that it also moves as we commit changes (similar to HEAD). The default branch is `master` branch. Creating branch in git is a very inexpensive operation and can be done by executing:
```sh
git branch feature_branch
```

To view all branches,
```sh
git branch
```
Or
```sh
git branch -v
```
for a more informative output.

To switch to the new branch we execute
```sh
git checkout feature_branch
```

The above code switches the current branch and replaces the working directory with HEAD (which now points to the latest commit of the new branch). Checkout may fail if we have some changes in either the working directory or the staging area. So we need to either commit or stash or use `--force` option.

![Git Branch Flow](https://i.imgur.com/t9Ar4FE.png)

We can delete a branch by
```sh
git branch -d feature_branch
```
Deleting a branch does not delete any commits on the branch but they lose the branch reference. If we know the SHA1 hash of the commits we can still checkout them. To rename a branch,
```sh
git branch -m feature_branch feature
```

**Detached HEAD:** whenever we checkout a commit, HEAD now points to that commit. This situation is called as detached HEAD state. If we commit now, that commit becomes the new HEAD. If we again do `git checkout master`, we come back to the previous head. However we lose reference to the new commit we did. So git hints us to create a new branch to represent that new commit which we can do by
```
git branch <branch_name> <SHA1>
```

To checkout a particular file,
```sh
git checkout 6b43a12 util.py
```

![Git Checkout Flow](https://i.imgur.com/GRFFfjU.png)

### Merging
To merge commit from branch `feature` into `master`, you follow the following steps:
```sh
git checkout master
git merge feature
```

Similar to how we can't checkout if there are uncommited changes in the working directory, you can't merge if there is a similar situation. So we need to `stash` those changes.  

Types of merges:
- **Fast-forward merge:** A fast-forward merge can happen if what you're trying to merge in already contains all of the content of the destination.
![Fast forward](https://i.imgur.com/nAP1Vug.png)  
- **Three-way merge:** In this scenario, a fast-forward is not possible because both branches to be merged have updates since they diverged from a common point. Those divergences have to be resolved before the merge can be completed.  
![3 way merge](https://i.imgur.com/2bFMpQo.png)  

Merge operation can be deleted by doing a `git reset`. **ORIG_HEAD** contains the SHA1 value of the commit that was current before the last merge operation. So we can do `git reset --hard ORIG_HEAD`.

**Dealing with merge conflicts:** One of the first things to understand is that merge operations are states in Git. This means that when the operations are started, Git enters a state that disallows changing contexts until the operation is complete. We can abort a merge operation by
```sh
git merge --abort
```

The situation of conflict is shown in the diagram below:  
![Merge Conflict](https://i.imgur.com/mhQ1JV9.png)  

For any files that have conflicts, those conflicts are marked with the Git conflict markers (≪≪≪ and ≫≫≫). Now, it is up to the user to resolve the conflicts that Git couldn't merge, and complete the operation. You cannot exit the merge state without resolving conflict in the affected file. `git status` lets us know which files have merge conflict. It is better to use a visual merge conflict resolve tool. In tortoise git the workflow is like:  

![TortoiseGitMerge](https://i.imgur.com/ZcbZKrB.png)